In [1]:
import numpy as np
import scipy.io
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
from scipy import stats

In [2]:
os.chdir('/mnt/fileserv1_neurologie/Research_groups/mgoerner/projects/scientific/monkey_gf/data')
details = pd.read_csv('all_data.csv') 

### EYE&HEAD

In [3]:
filename_lst = []
soas_lst = []
congr_perf_lst = []
incongr_perf_lst = []
congr_n = []
incongr_n = []
congr_RT_lst = []
incongr_RT_lst = []

In [4]:
os.chdir('/mnt/fileserv1_neurologie/Research_groups/mgoerner/projects/scientific/monkey_gf/data/eye_head/nrec')
final_df = pd.DataFrame({'filename': [], 'soa': [], 'perf_congr': [], 'perf_incongr': [], 'RT_congr': [], 'RT_incongr':[], 'n_congr': [], 'n_incongr':[]})

for file in os.listdir():
    if file in details['filename'].tolist():
        #print(file)
        mat_name = str(file)
    else:
        continue
        
    mat = scipy.io.loadmat(mat_name)
    session = details[details['filename']==mat_name]
    
    num_trials = mat['num_trials'][0][0]
    soa_lst = np.unique(session['soa'].tolist())
        
    valid = session[(session['response_from_tracking'] == 2) | (session['response_from_tracking'] == 6)].reset_index(drop=True)
    valid_trials = valid['trial_id'].tolist()
    invalid = session[(session['response_from_tracking'] != 2) & (session['response_from_tracking'] != 6)] 
    invalid_trials = invalid['trial_id'].tolist()
    
    correct = valid[valid['got_reward'] == 1]
    incorrect = valid[valid['got_reward'] != 1]
    incorrect_trials = incorrect['trial_id'].tolist()

    congruent = valid[valid['cue_lum_congruency'] == True] 
    congruent_trials = congruent['trial_id'].tolist()
    incongruent = valid[valid['cue_lum_congruency'] == False] 
    incongruent_trials = incongruent['trial_id'].tolist()
    
    valid_trials = [i for i in valid_trials if i not in invalid_trials]
    
    soi = [0.02, 0.05, 0.1, 0.2, 0.3] # soa of interest

    
    if len(correct)>10:
        for i in soa_lst:
            if i in soi:
                subset = valid[valid['soa'] == i]
                
                ## CONGRUENT
                corr_congr = subset[(subset['got_reward'] == 1) & (subset['cue_lum_congruency'] == True)]
                incorr_congr = subset[(subset['got_reward'] == 0) & (subset['cue_lum_congruency'] == True)]
                congr_n_trials = len(corr_congr) + len(incorr_congr)

                ## INCONGRUENT
                corr_incongr = subset[(subset['got_reward'] == 1) & (subset['cue_lum_congruency'] == False)]
                incorr_incongr = subset[(subset['got_reward'] == 0) & (subset['cue_lum_congruency'] == False)]
                incongr_n_trials = len(corr_incongr) + len(incorr_incongr)
                
                if len(incorr_incongr) > 5 and len(incorr_congr) > 5:
                    print(file)
                    filename_lst.append(file)
                    soas_lst.append(i)
                    congr_perf = len(corr_congr) / congr_n_trials
                    print(congr_n_trials)
                    congr_n.append(congr_n_trials)
                    congr_perf_lst.append(congr_perf)
                    print('Performance in congruent:', congr_perf, 'soa', i)
                    congr_RT =  np.mean(corr_congr['saccade_time_to_lum'])
                    congr_RT_lst.append(congr_RT)
                    incongr_perf = len(corr_incongr) / incongr_n_trials
                    incongr_n.append(incongr_n_trials)
                    incongr_perf_lst.append(incongr_perf)
                    print('Performance in incongruent:', incongr_perf, 'soa', i)
                    incongr_RT = np.mean(corr_incongr['saccade_time_to_lum'])
                    incongr_RT_lst.append(incongr_RT)
                    temp_df = pd.DataFrame({'filename':filename_lst, 'soa': soas_lst, 'perf_congr': congr_perf_lst, 'perf_incongr': incongr_perf_lst, 
                            'RT_congr': congr_RT_lst, 'RT_incongr': incongr_RT_lst, 'n_congr': congr_n, 'n_incongr':incongr_n})
                    final_df = pd.concat([final_df,temp_df])
                else:
                    continue
            else:
                continue

        else:
            continue


Emil_LED26_2022-10-28-13-19-50-247.mat
44
Performance in congruent: 0.8636363636363636 soa 0.05
Performance in incongruent: 0.7692307692307693 soa 0.05
Emil_LED26_2022-10-28-13-53-24-993.mat
81
Performance in congruent: 0.8395061728395061 soa 0.05
Performance in incongruent: 0.8289473684210527 soa 0.05
Emil_LED26_2022-10-28-13-53-24-993.mat
78
Performance in congruent: 0.782051282051282 soa 0.1
Performance in incongruent: 0.8076923076923077 soa 0.1
Emil_LED26_2022-10-28-13-53-24-993.mat
78
Performance in congruent: 0.6794871794871795 soa 0.3
Performance in incongruent: 0.6282051282051282 soa 0.3
Emil_LED26_2022-11-02-15-33-20-960.mat
62
Performance in congruent: 0.9032258064516129 soa 0.05
Performance in incongruent: 0.7936507936507936 soa 0.05
Emil_LED26_2022-11-02-15-33-20-960.mat
53
Performance in congruent: 0.8301886792452831 soa 0.1
Performance in incongruent: 0.6909090909090909 soa 0.1
Emil_LED26_2022-11-02-15-33-20-960.mat
51
Performance in congruent: 0.6666666666666666 soa 0.3


In [5]:
final_df.reset_index(drop=True)

,filename,soa,perf_congr,perf_incongr,RT_congr,RT_incongr,n_congr,n_incongr
0,Emil_LED26_2022-10-28-13-19-50-247.mat,0.05,0.863636,0.769231,484.302632,565.750000,44.0,39.0
1,Emil_LED26_2022-10-28-13-19-50-247.mat,0.05,0.863636,0.769231,484.302632,565.750000,44.0,39.0
2,Emil_LED26_2022-10-28-13-53-24-993.mat,0.05,0.839506,0.828947,499.330882,494.396825,81.0,76.0
3,Emil_LED26_2022-10-28-13-19-50-247.mat,0.05,0.863636,0.769231,484.302632,565.750000,44.0,39.0
4,Emil_LED26_2022-10-28-13-53-24-993.mat,0.05,0.839506,0.828947,499.330882,494.396825,81.0,76.0
...,...,...,...,...,...,...,...,...
4748,Emil_LED26_2023-04-12-10-34-58-107.mat,0.10,0.813953,0.743590,401.657143,413.275862,43.0,39.0
4749,Emil_LED26_2023-04-12-11-08-09-783.mat,0.10,0.774194,0.714286,466.770833,452.000000,31.0,21.0
4750,Emil_LED26_2023-04-14-15-04-24-354.mat,0.02,0.857143,0.767442,485.152778,558.227273,42.0,43.0
4751,Emil_LED26_2023-04-14-15-04-24-354.mat,0.05,0.840909,0.772727,466.972973,504.558824,44.0,44.0


In [6]:
final_df.to_csv('performance_eyehead.csv')

### EYE ONLY

In [7]:
filename_lst = []
soas_lst = []
congr_perf_lst = []
incongr_perf_lst = []
congr_n = []
incongr_n = []
congr_RT_lst = []
incongr_RT_lst = []

In [8]:
os.chdir('/mnt/fileserv1_neurologie/Research_groups/mgoerner/projects/scientific/monkey_gf/data/eye_only/nrec')
final_df = pd.DataFrame({'filename': [], 'soa': [], 'perf_congr': [], 'perf_incongr': [], 'RT_congr': [], 'RT_incongr':[], 'n_congr': [], 'n_incongr':[]})

for file in os.listdir():
    if file in details['filename'].tolist():
        #print(file)
        mat_name = str(file)
    else:
        continue
        
    mat = scipy.io.loadmat(mat_name)
    session = details[details['filename']==mat_name]
    
    num_trials = mat['num_trials'][0][0]
    soa_lst = np.unique(session['soa'].tolist())
        
    valid = session[(session['response_from_tracking'] == 2) | (session['response_from_tracking'] == 6)].reset_index(drop=True)
    valid_trials = valid['trial_id'].tolist()
    invalid = session[(session['response_from_tracking'] != 2) & (session['response_from_tracking'] != 6)] 
    invalid_trials = invalid['trial_id'].tolist()
    
    correct = valid[valid['got_reward'] == 1]
    incorrect = valid[valid['got_reward'] != 1]
    incorrect_trials = incorrect['trial_id'].tolist()

    congruent = valid[valid['cue_lum_congruency'] == True] 
    congruent_trials = congruent['trial_id'].tolist()
    incongruent = valid[valid['cue_lum_congruency'] == False] 
    incongruent_trials = incongruent['trial_id'].tolist()
    
    valid_trials = [i for i in valid_trials if i not in invalid_trials]
    
    soi = [0.02, 0.05, 0.1, 0.2, 0.3] # soa of interest

    
    if len(correct)>10:
        for i in soa_lst:
            if i in soi:
                subset = valid[valid['soa'] == i]
                
                ## CONGRUENT
                corr_congr = subset[(subset['got_reward'] == 1) & (subset['cue_lum_congruency'] == True)]
                incorr_congr = subset[(subset['got_reward'] == 0) & (subset['cue_lum_congruency'] == True)]
                congr_n_trials = len(corr_congr) + len(incorr_congr)

                ## INCONGRUENT
                corr_incongr = subset[(subset['got_reward'] == 1) & (subset['cue_lum_congruency'] == False)]
                incorr_incongr = subset[(subset['got_reward'] == 0) & (subset['cue_lum_congruency'] == False)]
                incongr_n_trials = len(corr_incongr) + len(incorr_incongr)
                
                if len(incorr_incongr) > 5 and len(incorr_congr) > 5:
                    print(file)
                    filename_lst.append(file)
                    soas_lst.append(i)
                    congr_perf = len(corr_congr) / congr_n_trials
                    print(congr_n_trials)
                    congr_n.append(congr_n_trials)
                    congr_perf_lst.append(congr_perf)
                    print('Performance in congruent:', congr_perf, 'soa', i)
                    congr_RT =  np.mean(corr_congr['saccade_time_to_lum'])
                    congr_RT_lst.append(congr_RT)
                    incongr_perf = len(corr_incongr) / incongr_n_trials
                    incongr_n.append(incongr_n_trials)
                    incongr_perf_lst.append(incongr_perf)
                    print('Performance in incongruent:', incongr_perf, 'soa', i)
                    incongr_RT = np.mean(corr_incongr['saccade_time_to_lum'])
                    incongr_RT_lst.append(incongr_RT)
                    temp_df = pd.DataFrame({'filename':filename_lst, 'soa': soas_lst, 'perf_congr': congr_perf_lst, 'perf_incongr': incongr_perf_lst, 
                            'RT_congr': congr_RT_lst, 'RT_incongr': incongr_RT_lst, 'n_congr': congr_n, 'n_incongr':incongr_n})
                    final_df = pd.concat([final_df,temp_df])
                else:
                    continue
            else:
                continue

        else:
            continue


Emil_LED26_2023-04-20-15-09-54-856.mat
39
Performance in congruent: 0.8205128205128205 soa 0.05
Performance in incongruent: 0.8421052631578947 soa 0.05
Emil_LED26_2023-04-20-15-09-54-856.mat
52
Performance in congruent: 0.7115384615384616 soa 0.1
Performance in incongruent: 0.46153846153846156 soa 0.1
Emil_LED26_2023-04-20-15-40-49-802.mat
56
Performance in congruent: 0.7678571428571429 soa 0.02
Performance in incongruent: 0.6842105263157895 soa 0.02
Emil_LED26_2023-04-20-15-40-49-802.mat
57
Performance in congruent: 0.7543859649122807 soa 0.05
Performance in incongruent: 0.5384615384615384 soa 0.05
Emil_LED26_2023-04-20-15-40-49-802.mat
62
Performance in congruent: 0.6774193548387096 soa 0.1
Performance in incongruent: 0.4430379746835443 soa 0.1
Emil_LED26_2023-04-24-17-00-22-311.mat
47
Performance in congruent: 0.8085106382978723 soa 0.02
Performance in incongruent: 0.875 soa 0.02
Emil_LED26_2023-04-24-17-00-22-311.mat
63
Performance in congruent: 0.8412698412698413 soa 0.05
Performa

In [9]:
final_df.reset_index(drop=True)

,filename,soa,perf_congr,perf_incongr,RT_congr,RT_incongr,n_congr,n_incongr
0,Emil_LED26_2023-04-20-15-09-54-856.mat,0.05,0.820513,0.842105,425.640625,389.578125,39.0,38.0
1,Emil_LED26_2023-04-20-15-09-54-856.mat,0.05,0.820513,0.842105,425.640625,389.578125,39.0,38.0
2,Emil_LED26_2023-04-20-15-09-54-856.mat,0.10,0.711538,0.461538,361.716216,385.458333,52.0,52.0
3,Emil_LED26_2023-04-20-15-09-54-856.mat,0.05,0.820513,0.842105,425.640625,389.578125,39.0,38.0
4,Emil_LED26_2023-04-20-15-09-54-856.mat,0.10,0.711538,0.461538,361.716216,385.458333,52.0,52.0
...,...,...,...,...,...,...,...,...
72005,Emil_LED26_2023-10-11-11-20-07-160.mat,0.05,0.350000,0.545455,368.071429,458.000000,20.0,22.0
72006,Emil_LED26_2023-10-11-11-20-07-160.mat,0.10,0.545455,0.500000,418.541667,542.937500,22.0,16.0
72007,Emil_LED26_2023-10-11-11-20-07-160.mat,0.30,0.272727,0.416667,190.583333,291.200000,22.0,24.0
72008,Emil_LED26_2023-10-11-11-28-36-466.mat,0.05,0.388889,0.333333,469.642857,443.785714,18.0,21.0


In [10]:
final_df.to_csv('performance_eyeonly.csv')